<table align="left">
    <tr>
        <td style="vertical-align: middle; padding-left: 0px; padding-right: 0px;">
            <a href="https://creativecommons.org/licenses/by/4.0/">
                <img src="https://licensebuttons.net/l/by/4.0/80x15.png" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 5px; padding-right: 0px;">
            <a href="https://opensource.org/licenses/MIT">
                <img src="https://img.shields.io/badge/License-MIT-green.svg" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 15px;">
            &copy; Guillaume Rongier
        </td>
    </tr>
</table>

# Adding sediment compaction

This notebook starts from the basic example of the [first notebook](1_basic-example.ipynb) to show how to use the component managing sediment compaction.

### Imports

Let's first import all the required packages and components:

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cmocean
import pyvista as pv

from landlab.components import FlowDirectorMFD, FlowAccumulator

from stratigrapy import RasterModelGrid
from stratigrapy.components import CyclicSeaLevelCalculator, WaterDrivenRouter, SedimentCompactor
from stratigrapy.plot import extract_tie_centered_layers

## 1. Compaction during simulation

We'll use the same case as in the [first notebook](1_basic-example.ipynb), starting with the same simulation time:

In [ ]:
timestep = 100.
runtime = 500000.
n_iterations = int(runtime/timestep)

Then the grid:

In [ ]:
grid = RasterModelGrid((25, 30),
                       xy_spacing=(2500., 2500.),
                       number_of_classes=2,
                       initial_allocation=n_iterations//100 + 100,
                       number_of_layers_to_fuse=100,
                       number_of_top_layers=100,
                       fuse_continuously=True)

In [ ]:
grid.set_closed_boundaries_at_grid_edges(True, True, True, False)

The initial topography:

In [ ]:
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation += 0.003*(grid.y_of_node - 50000.)

The sources of water and sediments:

In [ ]:
idx = np.ravel_multi_index(((23, 23), (14, 15)), grid.shape)
water_influx = grid.add_zeros('water__unit_flux_in', at='node', clobber=True)
water_influx[idx] = 5000. # m/yr

In [ ]:
sediment_influx = grid.add_field('sediment__unit_flux_in',
                                 np.zeros((grid.number_of_nodes, 2)),
                                 clobber=True)
sediment_influx[idx] = [0.7*50000., 0.3*50000.] # m3/yr

The component controlling sea level:

In [ ]:
slc = CyclicSeaLevelCalculator(grid, wavelength=[100000., 10000.], amplitude=[25., 2.5])

The component controlling water flow:

In [ ]:
fd = FlowDirectorMFD(grid, partition_method='slope', diagonals=True)

In [ ]:
fa = FlowAccumulator(grid, flow_director=fd)

Then we can define the component for sediment compaction:

In [ ]:
sc = SedimentCompactor(grid, initial_porosity=[0.6, 0.4], efolding_thickness=[1500., 2000.])

And the component controlling sediment transport, in which we need to add the sediment initial porosity throught the parameter `porosity` and an extra field to track, `sediment__porosity`, which is created by SedimentCompactor:

In [ ]:
wdr = WaterDrivenRouter(grid,
                        transportability_cont=[1e-8, 1e-8],
                        transportability_mar=[4e-10, 2e-10],
                        wave_base=15.,
                        porosity=[0.6, 0.4],
                        max_erosion_rate_sed=1e-2,
                        max_erosion_rate_br=1e-12,
                        bedrock_composition=[0.7, 0.3],
                        fields_to_track=['bathymetric__depth', 'sediment__porosity'])

Now we can run the simulation, adding the function to run the compaction and fusing the porosity with a mean weighted by layer thickness:

In [ ]:
for i in tqdm(range(n_iterations)):
    slc.run_one_step(timestep)
    fa.run_one_step()
    wdr.run_one_step(timestep)
    sc.run_one_step()
    grid.stacked_layers.fuse(time=np.mean, bathymetric__depth=np.mean, sediment__porosity='weighted_mean')
grid.stacked_layers.fuse(finalize=True, time=np.mean, bathymetric__depth=np.mean, sediment__porosity='weighted_mean')

Note that compaction significantly slows down the simulation. Let's visualize the result:

In [ ]:
fig, ax = plt.subplots()

raster_x = grid.x_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)

pc = ax.pcolormesh(raster_x, raster_y, raster_z, cmap=cmocean.cm.topo,
                   norm=mcolors.CenteredNorm(grid.at_grid['sea_level__elevation']))
fig.colorbar(pc, ax=ax, label='Elevation (m)')

ax.set(xlabel='x (m)', ylabel='y (m)', aspect='equal');

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3.5))

# Sediments
pc = grid.plot_layers(ax, 'composition', i_class=0, mask_wedges=True, cmap='pink', zorder=2)
fig.colorbar(pc[0], ax=ax, label='Fraction of the first sediment class')

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
# Sea level
fill_sea = ax.fill_between(raster_y, raster_z, grid.at_grid['sea_level__elevation'],
                           color='#c6dbef', zorder=0)
# Bedrock
ymin, ymax = ax.get_ylim()
ax.fill_between(raster_y, raster_z, ymin, color='#d9d9d9', zorder=1)

ax.set(xlabel='y (m)', ylabel='z (m)', ylim=(ymin, ymax));

We can also visualize the overall sediment porosity, using the keyword 'weighted_mean' for the parameter `i_class` to account for the fraction of each sediment class:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3.5))

# Sediments
pc = grid.plot_layers(ax, 'sediment__porosity', i_class='weighted_mean',
                      mask_wedges=True, mask_value=0., cmap='cividis', zorder=2)
fig.colorbar(pc[0], ax=ax, label='Porosity')

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
# Sea level
fill_sea = ax.fill_between(raster_y, raster_z, grid.at_grid['sea_level__elevation'],
                           color='#c6dbef', zorder=0)
# Bedrock
ymin, ymax = ax.get_ylim()
ax.fill_between(raster_y, raster_z, ymin, color='#d9d9d9', zorder=1)

ax.set(xlabel='y (m)', ylabel='z (m)', ylim=(ymin, ymax));

We can do the same to visualize the sediment porosity at the surface:

In [ ]:
fig, ax = plt.subplots()

pc = grid.plot_layers(ax, 'sediment__porosity', i_x=None, i_layer='top',
                      i_class='weighted_mean', mask_value=0.,
                      shading='nearest', cmap='cividis')
fig.colorbar(pc, ax=ax, label='Fraction of first lithology')

ax.set(xlabel='x (m)', ylabel='y (m)', aspect='equal');

## 2. Compaction after simulation

As noted by by [Granjeon (1996)](https://theses.hal.science/tel-00648827v1), compaction near the surface leads to few changes compared to the case without compaction (most of the visible changes here come from taking the sediment porosity into account during sediment transport and deposition). But the component can also be used to adjust the stratigraphy to greater depths. For instance, let's consider that the delta gets convered by a kilometer of sediments made of 70% of the first class and 20% of the second, and add that layer to the stratigraphy:

In [ ]:
grid.stacked_layers.add([0.7*1000., 0.3*1000.],
                        time=5100000.,
                        bathymetric__depth=0.,
                        sediment__porosity=[0.6, 0.4])

Now let's run the compaction only:

In [ ]:
sc.run_one_step()

And let's remove that extra layer to make visualization easier:

In [ ]:
grid.at_node['topographic__elevation'][grid.core_nodes] -= grid.stacked_layers.layer_thickness[-1]

In [ ]:
grid.stacked_layers.remove_last_layers()

Now we can clearly see the impact of the compaction:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3.5))

# Sediments
pc = grid.plot_layers(ax, 'composition', i_class=0, mask_wedges=True, cmap='pink', zorder=2)
fig.colorbar(pc[0], ax=ax, label='Fraction of the first sediment class')

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
# Overburden
ymin, ymax = ax.get_ylim()
ax.fill_between(raster_y, raster_z, ymax, color='0.97', zorder=0)
# Bedrock
ax.fill_between(raster_y, raster_z, ymin, color='#d9d9d9', zorder=1)

ax.set(xlabel='y (m)', ylabel='z (m)', ylim=(ymin, ymax));

Visualizing the porosity shows a layer of high porosity at depth, which comes from layer fusing and the fact that porosity in each layer is computed at the center of a layer:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3.5))

# Sediments
pc = grid.plot_layers(ax, 'sediment__porosity', i_class='weighted_mean',
                      mask_wedges=True, mask_value=0., cmap='cividis', zorder=2)
fig.colorbar(pc[0], ax=ax, label='Porosity')

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
# Overburden
ymin, ymax = ax.get_ylim()
ax.fill_between(raster_y, raster_z, ymax, color='0.97', zorder=0)
# Bedrock
ax.fill_between(raster_y, raster_z, ymin, color='#d9d9d9', zorder=1)

ax.set(xlabel='y (m)', ylabel='z (m)', ylim=(ymin, ymax));